# 🚀 LoRA 参数高效微调完整教程 - 面向初学者

## 📖 教程概述

本教程专为大模型微调初学者设计，通过一个完整的 **SMS 垃圾短信分类任务**，深入浅出地讲解 **LoRA（Low-Rank Adaptation）** 参数高效微调技术的**原理、实现和应用**。

### 🎯 你将学到什么

**理论层面**：
- ✅ **LoRA 核心原理**：为什么低秩分解能实现参数高效微调？
- ✅ **数学公式详解**：从线性代数角度理解 `ΔW ≈ A × B` 的含义
- ✅ **超参数调优**：rank、alpha 等关键参数如何影响微调效果？
- ✅ **优势对比**：相比全量微调，LoRA 的优势在哪里？

**实践层面**：
- ✅ **完整工作流**：从数据准备到模型部署的端到端流程
- ✅ **代码实现**：手把手实现 LoRA 层替换和训练
- ✅ **性能评估**：如何评估微调效果和可视化训练过程
- ✅ **实际应用**：将通用语言模型适配为垃圾短信分类器

### 🔍 LoRA 技术优势

| 对比维度 | 全量微调 | LoRA 微调 |
|---------|----------|----------|
| **训练参数量** | 全部模型参数（数十亿） | 仅 LoRA 参数（数百万） |
| **显存占用** | 非常高 | 显著降低 |
| **训练时间** | 较长 | 较短 |
| **部署成本** | 需要完整模型 | 只需增量参数 |
| **多任务适配** | 需要多个完整模型 | 共享基座+多个 LoRA |

### 🎓 预期学习效果

**性能指标**：
- 📊 **准确率提升**：从随机猜测（~50%）提升至 95%+
- ⚡ **训练效率**：3-5 个 epoch 即可收敛
- 💾 **参数效率**：仅训练 <1% 的模型参数
- 🚀 **部署友好**：LoRA 文件仅几 MB，便于分发

**技能掌握**：
- 🔧 理解并实现 LoRA 算法
- 📈 掌握模型训练和评估方法
- 🎯 能够将理论应用到实际项目中
- 💡 具备解决类似微调任务的能力

---

## 🗂️ 教程目录结构

```
📚 LoRA 微调教程
├── 🔧 1. 环境准备和依赖安装
├── 📖 2. LoRA 理论详解（核心原理 + 数学公式）
├── 📊 3. 数据集准备和预处理
├── 🤖 4. GPT-2 模型加载和适配
├── ⚡ 5. LoRA 层实现和替换
├── 🚀 6. 模型训练和优化
├── 📈 7. 性能评估和可视化
└── 🎯 8. 实际应用和部署
```

💡 **学习建议**：建议按顺序执行每个章节，理论和实践相结合，遇到问题及时查阅注释和文档。


# 🔧 第1章：环境准备和依赖安装

## 💻 运行环境要求

### 🖥️ 硬件要求
- **CPU**：支持现代指令集的多核处理器
- **内存**：建议 8GB+ RAM（16GB+ 更佳）
- **GPU**：推荐支持 CUDA 的 NVIDIA GPU（可选，但能显著加速训练）
- **存储**：至少 2GB 可用空间

### 🐍 软件要求
- **Python**：3.8+ 版本
- **操作系统**：Windows 10+/macOS 10.15+/Ubuntu 18.04+
- **Jupyter**：支持 Notebook 环境

### 📦 核心依赖包
我们将安装以下核心依赖包，每个都有其特定用途：

- 🔥 **PyTorch**：深度学习框架，提供张量计算和自动微分
- 🔤 **tiktoken**：OpenAI 的 BPE 分词器，用于 GPT-2 文本预处理
- 📊 **pandas**：数据处理库，用于 CSV 文件操作和数据分析
- 📈 **matplotlib**：可视化库，用于绘制训练曲线和结果图表
- 🔢 **numpy**：数值计算基础库，提供多维数组操作
- 🎨 **seaborn**：高级可视化库，提供美观的统计图表
- 📚 **transformers**：Hugging Face 模型库（辅助功能）
- ⏳ **tqdm**：进度条库，显示训练进度

## 🚀 快速环境检查


In [ ]:
# 🔍 简化版环境检查
# 功能：快速检查系统基本信息，确保满足运行要求

import platform
import sys

print("🔍 系统环境检查")
print("=" * 50)

# 检查操作系统信息
print(f"📱 操作系统: {platform.system()} {platform.release()}")
print(f"🐍 Python 版本: {sys.version.split()[0]}")

# 检查 PyTorch 和 CUDA 支持
try:
    import torch
    print(f"🔥 PyTorch 版本: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"🎮 GPU 支持: ✅ (CUDA {torch.version.cuda})")
        print(f"🎮 GPU 数量: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"   - GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("🎮 GPU 支持: ❌ (将使用CPU，训练速度较慢)")
except ImportError:
    print("❌ PyTorch 未安装，请先安装依赖包")

print("\n✅ 环境检查完成！")


## 📦 一键安装所有依赖

以下命令将安装所有必需的依赖包。我们指定了具体版本号以确保兼容性和结果的可重现性。


In [ ]:
# 📦 核心依赖一键安装
# 说明：这里安装的是经过测试的稳定版本组合，确保兼容性

print("🚀 开始安装 LoRA 微调所需的核心依赖包")
print("⏳ 预计安装时间：2-5分钟（取决于网络速度）")
print("=" * 60)

# 安装 PyTorch（支持 CUDA）
%pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118

# 安装其他核心依赖
%pip install numpy==1.24.3 pandas==2.0.3 matplotlib==3.7.2 seaborn==0.12.2 tiktoken==0.5.1 transformers==4.33.2 tqdm==4.66.1 requests==2.31.0

print("\n✅ 所有依赖包安装完成！")
print("💡 建议重启 Jupyter Kernel 以确保新安装的包正常工作")


# 📖 第2章：LoRA 理论详解 - 从零理解核心原理

## 🎯 什么是 LoRA？

**LoRA (Low-Rank Adaptation)** 是一种参数高效的微调技术，由微软研究院在 2021 年提出。它通过**低秩矩阵分解**的方法，在保持原始模型参数不变的情况下，仅训练少量的增量参数来适配新任务。

### 🔍 核心思想

传统微调需要更新模型的**所有参数**，而 LoRA 的创新之处在于：
1. **冻结原始参数**：保持预训练模型的权重 W 不变
2. **低秩分解**：将权重更新 ΔW 分解为两个更小的矩阵 A 和 B
3. **并行计算**：在推理时将原始输出与 LoRA 分支的输出相加

## 🧮 数学原理详解

### 📐 基础公式

在传统的全量微调中，线性层的前向传播为：
```
h = x · W
```

其中：
- `x` ∈ ℝ^(d) 是输入向量
- `W` ∈ ℝ^(d×k) 是权重矩阵
- `h` ∈ ℝ^(k) 是输出向量

微调后的权重变为：
```
W_new = W + ΔW
```

### 🔑 LoRA 的核心创新

LoRA 将权重更新 ΔW 分解为两个低秩矩阵的乘积：

```
ΔW = A · B
```

其中：
- `A` ∈ ℝ^(d×r)（下投影矩阵）
- `B` ∈ ℝ^(r×k)（上投影矩阵）  
- `r` << min(d, k)（秩，远小于原始维度）

因此，LoRA 的前向传播变为：
```
h = x · W + α · (x · A · B)
```

其中 `α` 是缩放因子。

### 💡 通俗理解

想象你有一个巨大的**魔方（原始模型）**：

1. **传统微调**：需要重新调整魔方的每一个小块（所有参数）
2. **LoRA 微调**：只需要在魔方表面贴上特殊的**贴纸（LoRA 层）**，通过调整贴纸来改变整体效果

这样做的好处：
- 🎯 **效率高**：只需训练贴纸，不动原魔方
- 💾 **存储省**：只需保存贴纸的信息
- 🔄 **可切换**：可以快速更换不同的贴纸适配不同任务

## 📊 参数量对比

假设一个线性层的维度为 4096×4096：

| 方法 | 参数量 | 相对比例 |
|------|--------|----------|
| **全量微调** | 16,777,216 | 100% |
| **LoRA (r=16)** | 131,072 | 0.78% |
| **LoRA (r=32)** | 262,144 | 1.56% |
| **LoRA (r=64)** | 524,288 | 3.13% |

可以看到，即使是 r=64 的 LoRA，参数量也不到原始的 4%！


# 📊 第3章：数据集准备与预处理

## 🎯 数据集选择

我们使用经典的 **SMS Spam Collection** 数据集来演示 LoRA 微调：

### 📋 数据集信息
- **数据来源**：UCI Machine Learning Repository
- **数据规模**：约 5,572 条短信
- **任务类型**：二分类（正常短信 vs 垃圾短信）
- **标签分布**：
  - `ham`（正常短信）：约 87%
  - `spam`（垃圾短信）：约 13%

### 🔄 数据流程图

```
原始数据 → 类别平衡 → 数据划分 → 分词编码 → 批次加载
   ↓           ↓          ↓          ↓          ↓
 5572条    均衡采样    7:2:1比例   GPT-2编码   DataLoader
```

## 🛠️ 数据预处理核心函数


In [ ]:
# 🔧 数据处理工具函数库
# 功能：提供完整的数据预处理、模型训练和评估工具

import os
import requests
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import tiktoken

def create_balanced_dataset(df):
    """
    创建类别平衡的数据集
    
    作用：解决数据不平衡问题，避免模型偏向多数类
    
    参数：
        df: 包含 Label 和 Text 列的原始数据框
        
    返回：
        balanced_df: 类别平衡后的数据框
        
    示例：
        原始数据：ham(4827条) + spam(747条) = 5574条
        平衡后：ham(747条) + spam(747条) = 1494条
    """
    # 统计垃圾短信数量（少数类）
    num_spam = df[df["Label"] == "spam"].shape[0]
    print(f"📊 原始数据分布：")
    print(f"   - 正常短信(ham): {df[df['Label'] == 'ham'].shape[0]} 条")
    print(f"   - 垃圾短信(spam): {num_spam} 条")
    
    # 随机采样等量的正常短信（固定随机种子确保可复现）
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)
    
    # 合并平衡数据
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])
    print(f"🎯 平衡后数据：每类 {num_spam} 条，总计 {len(balanced_df)} 条")
    
    return balanced_df

def random_split(df, train_frac, validation_frac):
    """
    随机划分数据集
    
    参数：
        df: 待划分的数据框
        train_frac: 训练集比例 (如 0.7)
        validation_frac: 验证集比例 (如 0.2)
        
    返回：
        train_df, validation_df, test_df: 三个数据框
        
    示例：
        总数据1494条 → 训练1045条(70%) + 验证149条(10%) + 测试300条(20%)
    """
    # 随机打乱数据（固定随机种子确保可复现）
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)
    
    # 计算划分索引
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)
    
    # 执行划分
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]
    
    print(f"📊 数据划分结果：")
    print(f"   - 训练集: {len(train_df)} 条 ({len(train_df)/len(df)*100:.1f}%)")
    print(f"   - 验证集: {len(validation_df)} 条 ({len(validation_df)/len(df)*100:.1f}%)")
    print(f"   - 测试集: {len(test_df)} 条 ({len(test_df)/len(df)*100:.1f}%)")
    
    return train_df, validation_df, test_df

class SpamDataset(Dataset):
    """
    SMS 垃圾短信数据集类
    
    功能：
    1. 继承 PyTorch Dataset，支持 DataLoader 批量加载
    2. 自动处理文本分词、序列填充和标签转换
    3. 支持动态序列长度或固定长度截断
    
    使用流程：
    text → tokenize → padding → tensor
    """
    
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        """
        初始化数据集
        
        参数：
            csv_file: CSV 文件路径
            tokenizer: 分词器对象（tiktoken）
            max_length: 最大序列长度，None 时使用训练集最长长度
            pad_token_id: 填充 token ID（GPT-2 默认为 50256）
        """
        self.data = pd.read_csv(csv_file)
        print(f"📂 加载数据：{csv_file}，共 {len(self.data)} 条")
        
        # 预分词：将所有文本转换为 token ID 序列
        print("🔤 执行分词...")
        self.encoded_texts = [
            tokenizer.encode(text) for text in tqdm(self.data["Text"], desc="分词进度")
        ]
        
        # 确定序列长度
        if max_length is None:
            self.max_length = self._longest_encoded_length()
            print(f"📏 自动确定最大长度: {self.max_length}")
        else:
            self.max_length = max_length
            print(f"📏 使用指定长度: {self.max_length}")
            # 截断过长的序列
            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]
        
        # 填充序列到统一长度
        print("📐 执行填充...")
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]
        print("✅ 数据预处理完成")
    
    def __getitem__(self, index):
        """获取单个样本"""
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),  # 输入序列
            torch.tensor(label, dtype=torch.long)     # 标签
        )
    
    def __len__(self):
        """返回数据集大小"""
        return len(self.data)
    
    def _longest_encoded_length(self):
        """计算最长编码序列长度"""
        max_length = 0
        for encoded_text in self.encoded_texts:
            if len(encoded_text) > max_length:
                max_length = len(encoded_text)
        return max_length


# ⚡ 第4章：LoRA 层实现 - 核心算法

## 🧠 LoRA 层的 Python 实现

LoRA 的核心在于将原始的线性层替换为包含低秩分解的增强层。以下是完整的实现：


In [ ]:
# 🔧 LoRA 层完整实现
# 功能：实现参数高效的 LoRA 微调层

class LoRALayer(nn.Module):
    """
    LoRA (Low-Rank Adaptation) 层实现
    
    核心公式：h = x·W + α·(x·A·B)
    
    参数：
        - W: 冻结的原始权重 [d×k]
        - A: 下投影矩阵 [d×r] (可训练)
        - B: 上投影矩阵 [r×k] (可训练)  
        - α: 缩放因子
        - r: 低秩维度 (rank)
    """
    
    def __init__(self, in_dim, out_dim, rank, alpha):
        """
        初始化 LoRA 层
        
        参数:
            in_dim: 输入维度 (d)
            out_dim: 输出维度 (k)  
            rank: LoRA 秩 (r)
            alpha: 缩放因子
        """
        super().__init__()
        
        # 设置 LoRA 参数
        self.rank = rank
        self.alpha = alpha
        
        # LoRA 分支矩阵 (可训练参数)
        self.A = nn.Parameter(torch.randn(in_dim, rank) * 0.01)  # 小随机初始化
        self.B = nn.Parameter(torch.zeros(rank, out_dim))        # 零初始化确保初始时 ΔW=0
        
        print(f"✨ 创建 LoRA 层: {in_dim}×{out_dim} → rank={rank}, α={alpha}")
        print(f"   - A矩阵: {self.A.shape} ({self.A.numel():,} 参数)")
        print(f"   - B矩阵: {self.B.shape} ({self.B.numel():,} 参数)")
        print(f"   - 总参数: {self.A.numel() + self.B.numel():,}")
    
    def forward(self, x):
        """
        LoRA 前向传播
        
        计算: ΔW·x = α·(A·B)·x = α·x·A·B
        """
        # x·A: [batch, seq, in_dim] × [in_dim, rank] → [batch, seq, rank]
        # (x·A)·B: [batch, seq, rank] × [rank, out_dim] → [batch, seq, out_dim]
        lora_output = torch.matmul(torch.matmul(x, self.A), self.B)
        return self.alpha * lora_output

class LinearWithLoRA(nn.Module):
    """
    集成 LoRA 的线性层
    
    功能：将原始线性层与 LoRA 分支并行计算并相加
    前向传播：output = x·W + LoRA(x)
    """
    
    def __init__(self, linear_layer, rank, alpha):
        """
        参数:
            linear_layer: 原始的 nn.Linear 层
            rank: LoRA 秩
            alpha: LoRA 缩放因子
        """
        super().__init__()
        
        # 保存原始线性层 (冻结参数)
        self.linear = linear_layer
        for param in self.linear.parameters():
            param.requires_grad = False  # 冻结原始参数
        
        # 添加 LoRA 分支
        self.lora = LoRALayer(
            in_dim=linear_layer.in_features,
            out_dim=linear_layer.out_features, 
            rank=rank,
            alpha=alpha
        )
    
    def forward(self, x):
        """
        前向传播：原始输出 + LoRA 增量
        """
        # 原始线性层输出（冻结权重）
        original_output = self.linear(x)
        
        # LoRA 分支输出（可训练参数）
        lora_output = self.lora(x)
        
        # 返回叠加结果
        return original_output + lora_output

def replace_linear_with_lora(model, rank=16, alpha=32):
    """
    递归替换模型中所有 Linear 层为 LoRA 增强版本
    
    参数:
        model: 待替换的模型
        rank: LoRA 秩 (默认16)
        alpha: LoRA 缩放因子 (默认32，通常设为2×rank)
    
    返回:
        替换后的模型，原始参数被冻结，仅 LoRA 参数可训练
    """
    print(f"🔄 开始 LoRA 层替换 (rank={rank}, alpha={alpha})")
    
    total_replaced = 0
    original_params = sum(p.numel() for p in model.parameters())
    
    # 递归遍历所有子模块
    for name, module in model.named_children():
        if isinstance(module, nn.Linear):
            # 替换 Linear 层
            print(f"  📌 替换线性层: {name} ({module.in_features}×{module.out_features})")
            setattr(model, name, LinearWithLoRA(module, rank, alpha))
            total_replaced += 1
        else:
            # 递归处理子模块
            replace_linear_with_lora(module, rank, alpha)
    
    # 统计参数信息
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)
    
    print(f"\n📊 LoRA 替换完成统计:")
    print(f"   - 替换线性层数量: {total_replaced}")
    print(f"   - 原始参数总量: {original_params:,}")
    print(f"   - 冻结参数数量: {frozen_params:,}")
    print(f"   - 可训练参数数量: {trainable_params:,}")
    print(f"   - 参数效率: {trainable_params/original_params*100:.2f}%")
    
    return model


# 🤖 第5章：简化 GPT-2 模型实现

## 📚 轻量级 GPT-2 用于分类任务


In [ ]:
# 🏗️ 简化的 GPT-2 分类模型
# 功能：专门为二分类任务设计的轻量级 Transformer 模型

import math

class MultiHeadAttention(nn.Module):
    """多头注意力机制"""
    
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def forward(self, x):
        batch_size, seq_len, d_model = x.shape
        
        # 线性投影
        Q = self.W_q(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        
        # 缩放点积注意力
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # 因果掩码 (下三角)
        mask = torch.tril(torch.ones(seq_len, seq_len, device=x.device))
        scores.masked_fill_(mask == 0, float('-inf'))
        
        attn_weights = torch.softmax(scores, dim=-1)
        attn_output = torch.matmul(attn_weights, V)
        
        # 重新组织并输出投影
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, d_model)
        return self.W_o(attn_output)

class FeedForward(nn.Module):
    """前馈神经网络"""
    
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.gelu = nn.GELU()
        
    def forward(self, x):
        return self.linear2(self.gelu(self.linear1(x)))

class TransformerBlock(nn.Module):
    """Transformer 块"""
    
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        
    def forward(self, x):
        # 残差连接 + 层归一化
        x = x + self.attention(self.ln1(x))
        x = x + self.feed_forward(self.ln2(x))
        return x

class SimpleGPTForClassification(nn.Module):
    """
    简化的 GPT 分类模型
    
    特点：
    1. 基于 Transformer 架构
    2. 专门用于序列分类任务
    3. 使用最后一个 token 的表示进行分类
    4. 轻量级设计，便于 LoRA 微调演示
    """
    
    def __init__(self, vocab_size=50257, max_seq_len=512, d_model=256, 
                 num_heads=8, num_layers=6, d_ff=1024, num_classes=2):
        """
        参数：
            vocab_size: 词汇表大小 (GPT-2 默认 50257)
            max_seq_len: 最大序列长度
            d_model: 模型维度
            num_heads: 注意力头数
            num_layers: Transformer 层数
            d_ff: 前馈网络隐藏维度
            num_classes: 分类类别数 (二分类=2)
        """
        super().__init__()
        
        # Token 和位置嵌入
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_embedding = nn.Embedding(max_seq_len, d_model)
        
        # Transformer 层堆栈
        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff) 
            for _ in range(num_layers)
        ])
        
        # 最终层归一化
        self.ln_final = nn.LayerNorm(d_model)
        
        # 分类头 (用最后一个 token 进行分类)
        self.classifier = nn.Linear(d_model, num_classes)
        
        # 保存配置
        self.max_seq_len = max_seq_len
        self.d_model = d_model
        
        print(f"🤖 创建简化 GPT 分类模型:")
        print(f"   - 词汇表大小: {vocab_size:,}")
        print(f"   - 最大序列长度: {max_seq_len}")
        print(f"   - 模型维度: {d_model}")
        print(f"   - 注意力头数: {num_heads}")
        print(f"   - Transformer 层数: {num_layers}")
        print(f"   - 分类类别数: {num_classes}")
        
        # 统计参数量
        total_params = sum(p.numel() for p in self.parameters())
        print(f"   - 总参数量: {total_params:,}")
    
    def forward(self, input_ids):
        """
        前向传播
        
        参数：
            input_ids: [batch_size, seq_len] 输入 token ID
            
        返回：
            logits: [batch_size, num_classes] 分类 logits
        """
        batch_size, seq_len = input_ids.shape
        
        # 位置索引
        position_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0)
        
        # 嵌入
        token_emb = self.token_embedding(input_ids)        # [B, T, D]
        pos_emb = self.position_embedding(position_ids)    # [1, T, D]
        x = token_emb + pos_emb                           # [B, T, D]
        
        # Transformer 层
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x)
        
        # 最终层归一化
        x = self.ln_final(x)  # [B, T, D]
        
        # 使用最后一个 token 进行分类
        last_token_emb = x[:, -1, :]  # [B, D]
        
        # 分类头
        logits = self.classifier(last_token_emb)  # [B, num_classes]
        
        return logits

def create_model_for_classification(vocab_size=50257, max_seq_len=256):
    """
    创建用于分类的简化 GPT 模型
    
    参数调优说明：
    - d_model=256: 适中的模型维度，平衡性能和计算效率
    - num_heads=8: 多头注意力，每头维度 256/8=32
    - num_layers=4: 较少的层数，降低复杂度
    - d_ff=512: 前馈网络维度，通常为 d_model 的 2-4 倍
    """
    model = SimpleGPTForClassification(
        vocab_size=vocab_size,
        max_seq_len=max_seq_len,
        d_model=256,
        num_heads=8,
        num_layers=4,  # 减少层数以加快训练
        d_ff=512,
        num_classes=2
    )
    
    # 参数初始化 (Xavier/Glorot 初始化)
    for param in model.parameters():
        if param.dim() > 1:
            nn.init.xavier_uniform_(param)
    
    return model


# 🚀 第6章：训练和评估函数

## 📈 模型训练和性能评估工具


In [ ]:
# ⚡ 训练和评估核心函数
# 功能：提供完整的模型训练、评估和可视化工具

def calc_accuracy(model, data_loader, device, max_batches=None):
    """
    计算模型在数据集上的准确率
    
    参数：
        model: 待评估的模型
        data_loader: 数据加载器
        device: 计算设备
        max_batches: 最大评估批次数（None=全部）
        
    返回：
        accuracy: 准确率 (0-1 之间)
    """
    model.eval()
    correct, total = 0, 0
    
    with torch.no_grad():
        for i, (input_ids, labels) in enumerate(data_loader):
            if max_batches and i >= max_batches:
                break
                
            input_ids, labels = input_ids.to(device), labels.to(device)
            
            # 获取模型预测
            logits = model(input_ids)
            predictions = torch.argmax(logits, dim=-1)
            
            # 统计正确预测数
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
    model.train()
    return correct / total if total > 0 else 0.0

def calc_loss_batch(model, input_ids, labels, device):
    """计算单个批次的损失"""
    input_ids, labels = input_ids.to(device), labels.to(device)
    logits = model(input_ids)
    return nn.functional.cross_entropy(logits, labels)

def calc_loss_loader(model, data_loader, device, max_batches=None):
    """计算数据加载器的平均损失"""
    model.eval()
    total_loss = 0.0
    num_batches = 0
    
    with torch.no_grad():
        for i, (input_ids, labels) in enumerate(data_loader):
            if max_batches and i >= max_batches:
                break
            
            loss = calc_loss_batch(model, input_ids, labels, device)
            total_loss += loss.item()
            num_batches += 1
    
    model.train()
    return total_loss / num_batches if num_batches > 0 else float('inf')

def train_lora_model(model, train_loader, val_loader, device, num_epochs=3, 
                    learning_rate=3e-4, eval_every=50):
    """
    LoRA 模型训练主函数
    
    参数：
        model: LoRA 增强的模型
        train_loader: 训练数据加载器
        val_loader: 验证数据加载器
        device: 计算设备
        num_epochs: 训练轮数
        learning_rate: 学习率
        eval_every: 每多少步评估一次
        
    返回：
        训练历史记录 (损失、准确率等)
    """
    print(f"🚀 开始 LoRA 微调训练")
    print(f"   - 训练轮数: {num_epochs}")
    print(f"   - 学习率: {learning_rate}")
    print(f"   - 设备: {device}")
    
    # 移动模型到设备
    model.to(device)
    
    # 创建优化器 (仅优化可训练参数)
    trainable_params = [p for p in model.parameters() if p.requires_grad]
    print(f"   - 可训练参数: {sum(p.numel() for p in trainable_params):,}")
    
    optimizer = torch.optim.AdamW(trainable_params, lr=learning_rate)
    
    # 训练历史记录
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    steps = []
    
    global_step = 0
    
    # 训练循环
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        
        print(f"\n📚 Epoch {epoch + 1}/{num_epochs}")
        
        # 使用 tqdm 显示训练进度
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        
        for batch_idx, (input_ids, labels) in enumerate(pbar):
            # 前向传播
            optimizer.zero_grad()
            loss = calc_loss_batch(model, input_ids, labels, device)
            
            # 反向传播和优化
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            global_step += 1
            
            # 更新进度条
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            # 周期性评估
            if global_step % eval_every == 0:
                train_loss = calc_loss_loader(model, train_loader, device, max_batches=10)
                val_loss = calc_loss_loader(model, val_loader, device)
                
                train_acc = calc_accuracy(model, train_loader, device, max_batches=10)
                val_acc = calc_accuracy(model, val_loader, device)
                
                # 记录历史
                steps.append(global_step)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                train_accs.append(train_acc)
                val_accs.append(val_acc)
                
                print(f\"\\n   Step {global_step}: Train Loss={train_loss:.3f}, Val Loss={val_loss:.3f}\")\n                print(f\"   Train Acc={train_acc*100:.1f}%, Val Acc={val_acc*100:.1f}%\")\n        \n        # Epoch 结束评估\n        avg_epoch_loss = epoch_loss / len(train_loader)\n        print(f\"📊 Epoch {epoch+1} 完成 - 平均损失: {avg_epoch_loss:.4f}\")\n    \n    print(\"\\n🎉 LoRA 微调训练完成！\")\n    \n    return {\n        'steps': steps,\n        'train_losses': train_losses,\n        'val_losses': val_losses,\n        'train_accs': train_accs,\n        'val_accs': val_accs\n    }\n\ndef plot_training_curves(history, save_path='training_curves.png'):\n    \"\"\"\n    绘制训练曲线\n    \n    参数：\n        history: 训练历史记录\n        save_path: 保存路径\n    \"\"\"\n    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))\n    \n    steps = history['steps']\n    \n    # 损失曲线\n    ax1.plot(steps, history['train_losses'], 'b-', label='训练损失')\n    ax1.plot(steps, history['val_losses'], 'r--', label='验证损失')\n    ax1.set_xlabel('训练步数')\n    ax1.set_ylabel('损失')\n    ax1.set_title('损失曲线')\n    ax1.legend()\n    ax1.grid(True)\n    \n    # 准确率曲线\n    ax2.plot(steps, [acc*100 for acc in history['train_accs']], 'b-', label='训练准确率')\n    ax2.plot(steps, [acc*100 for acc in history['val_accs']], 'r--', label='验证准确率')\n    ax2.set_xlabel('训练步数')\n    ax2.set_ylabel('准确率 (%)')\n    ax2.set_title('准确率曲线')\n    ax2.legend()\n    ax2.grid(True)\n    \n    # 损失对比 (最后几个点)\n    if len(steps) > 5:\n        recent_steps = steps[-5:]\n        ax3.bar(range(len(recent_steps)), [history['train_losses'][i] for i in range(-5, 0)], \n               alpha=0.7, label='训练损失')\n        ax3.bar(range(len(recent_steps)), [history['val_losses'][i] for i in range(-5, 0)], \n               alpha=0.7, label='验证损失')\n        ax3.set_xlabel('最近5个评估点')\n        ax3.set_ylabel('损失')\n        ax3.set_title('最近损失对比')\n        ax3.legend()\n    \n    # 准确率提升\n    if len(history['val_accs']) > 1:\n        acc_improvement = [(acc - history['val_accs'][0])*100 for acc in history['val_accs']]\n        ax4.plot(steps, acc_improvement, 'g-', linewidth=2)\n        ax4.axhline(y=0, color='k', linestyle='--', alpha=0.5)\n        ax4.set_xlabel('训练步数')\n        ax4.set_ylabel('准确率提升 (%)')\n        ax4.set_title('验证准确率提升')\n        ax4.grid(True)\n    \n    plt.tight_layout()\n    plt.savefig(save_path, dpi=300, bbox_inches='tight')\n    plt.show()\n    \n    # 打印最终结果\n    if history['val_accs']:\n        print(f\"\\n📊 训练结果总结:\")\n        print(f\"   - 初始验证准确率: {history['val_accs'][0]*100:.2f}%\")\n        print(f\"   - 最终验证准确率: {history['val_accs'][-1]*100:.2f}%\")\n        print(f\"   - 准确率提升: {(history['val_accs'][-1] - history['val_accs'][0])*100:.2f} 百分点\")\n        print(f\"   - 最终训练损失: {history['train_losses'][-1]:.4f}\")\n        print(f\"   - 最终验证损失: {history['val_losses'][-1]:.4f}\")\n\ndef classify_text(text, model, tokenizer, device, max_length=256):\n    \"\"\"\n    对单条文本进行分类预测\n    \n    参数：\n        text: 待分类的文本\n        model: 训练好的模型\n        tokenizer: 分词器\n        device: 计算设备\n        max_length: 最大序列长度\n        \n    返回：\n        prediction: 预测结果 ('spam' 或 'ham')\n        confidence: 置信度\n    \"\"\"\n    model.eval()\n    \n    # 文本预处理\n    input_ids = tokenizer.encode(text)\n    if len(input_ids) > max_length:\n        input_ids = input_ids[:max_length]\n    else:\n        input_ids = input_ids + [50256] * (max_length - len(input_ids))  # 填充\n    \n    # 转为张量\n    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0)\n    \n    # 模型推理\n    with torch.no_grad():\n        logits = model(input_tensor)\n        probabilities = torch.softmax(logits, dim=-1)\n        predicted_class = torch.argmax(logits, dim=-1).item()\n        confidence = probabilities[0, predicted_class].item()\n    \n    # 返回结果\n    prediction = \"spam\" if predicted_class == 1 else \"ham\"\n    return prediction, confidence


# 🎯 第7章：完整 LoRA 微调实验流程

## 🚀 端到端实验：从数据到模型部署

现在让我们将所有组件整合起来，执行完整的 LoRA 微调实验。


In [ ]:
# 🧪 完整 LoRA 微调实验
# 功能：展示从数据准备到模型训练的完整流程

print("🔬 开始 LoRA 微调完整实验")
print("=" * 60)

# 设置实验参数
torch.manual_seed(42)  # 固定随机种子
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎮 使用设备: {device}")

# 1️⃣ 准备示例数据 (简化版本，无需下载)
print("\\n1️⃣ 准备示例数据...")

# 创建示例SMS数据集
sample_data = {
    'Label': ['ham', 'spam', 'ham', 'spam', 'ham', 'spam'] * 20,
    'Text': [
        'Hey, how are you doing today?',
        'WINNER! You have won $1000! Click here to claim!',
        'Can you pick up milk on your way home?',
        'FREE! Get rich quick scheme! No risk!',
        'Meeting is at 3pm in conference room B',
        'Congratulations! You are our lucky winner! Call now!'
    ] * 20
}

# 创建DataFrame并保存
df = pd.DataFrame(sample_data)
df['Label'] = df['Label'].map({'ham': 0, 'spam': 1})

# 平衡数据集
balanced_df = create_balanced_dataset(df)
train_df, val_df, test_df = random_split(balanced_df, 0.7, 0.1)

# 保存到CSV
train_df.to_csv('train_sample.csv', index=False)
val_df.to_csv('val_sample.csv', index=False)
test_df.to_csv('test_sample.csv', index=False)

# 2️⃣ 创建数据加载器
print("\\n2️⃣ 创建数据加载器...")
tokenizer = tiktoken.get_encoding("gpt2")

train_dataset = SpamDataset('train_sample.csv', tokenizer, max_length=64)
val_dataset = SpamDataset('val_sample.csv', tokenizer, max_length=train_dataset.max_length)
test_dataset = SpamDataset('test_sample.csv', tokenizer, max_length=train_dataset.max_length)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# 3️⃣ 创建模型
print("\\n3️⃣ 创建基础模型...")
model = create_model_for_classification(max_seq_len=train_dataset.max_length)

# 4️⃣ 应用 LoRA
print("\\n4️⃣ 应用 LoRA 微调...")
lora_model = replace_linear_with_lora(model, rank=8, alpha=16)

# 5️⃣ 训练模型
print("\\n5️⃣ 开始训练...")

# 简化训练函数
def quick_train(model, train_loader, val_loader, device, epochs=2):
    model.to(device)
    optimizer = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=1e-3)
    
    history = {'train_losses': [], 'val_losses': [], 'train_accs': [], 'val_accs': []}
    
    for epoch in range(epochs):
        # 训练
        model.train()
        train_loss = 0
        for input_ids, labels in train_loader:
            input_ids, labels = input_ids.to(device), labels.to(device)
            optimizer.zero_grad()
            logits = model(input_ids)
            loss = nn.functional.cross_entropy(logits, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # 评估
        train_acc = calc_accuracy(model, train_loader, device)
        val_acc = calc_accuracy(model, val_loader, device)
        val_loss = calc_loss_loader(model, val_loader, device)
        
        history['train_losses'].append(train_loss/len(train_loader))
        history['val_losses'].append(val_loss)
        history['train_accs'].append(train_acc)
        history['val_accs'].append(val_acc)
        
        print(f"Epoch {epoch+1}: Train Loss={train_loss/len(train_loader):.3f}, "
              f"Train Acc={train_acc*100:.1f}%, Val Acc={val_acc*100:.1f}%")
    
    return history

# 执行训练
history = quick_train(lora_model, train_loader, val_loader, device, epochs=3)

# 6️⃣ 测试模型
print("\\n6️⃣ 模型测试...")
test_acc = calc_accuracy(lora_model, test_loader, device)
print(f"🎯 最终测试准确率: {test_acc*100:.2f}%")

# 7️⃣ 实际应用演示
print("\\n7️⃣ 实际应用演示...")
test_texts = [
    "Hey, want to grab lunch together?",
    "URGENT! You've won $5000! Click now to claim your prize!",
    "The meeting has been moved to 2pm",
    "Free iPhone! Limited time offer! Call immediately!"
]

for text in test_texts:
    pred, conf = classify_text(text, lora_model, tokenizer, device, max_length=64)
    print(f"文本: '{text[:40]}...'")
    print(f"预测: {pred} (置信度: {conf:.3f})\\n")

print("🎉 LoRA 微调实验完成！")


# 🎓 总结与关键要点

## ✨ 本教程的核心收获

通过本教程，你已经掌握了 LoRA（Low-Rank Adaptation）微调的完整理论和实践：

### 🧠 理论理解
1. **LoRA 原理**：通过低秩矩阵分解 `ΔW ≈ A·B` 实现参数高效微调
2. **数学本质**：`h = x·W + α·(x·A·B)` - 原始输出叠加 LoRA 增量
3. **参数优势**：仅训练 <5% 参数，显著降低计算和存储成本
4. **初始化策略**：B矩阵零初始化确保训练开始时模型行为不变

### 💻 实践技能
1. **数据预处理**：类别平衡、序列填充、批量加载
2. **模型改造**：递归替换线性层为 LoRA 增强版本
3. **训练流程**：冻结原始参数，仅优化 LoRA 分支
4. **性能评估**：准确率计算、损失可视化、实际应用测试

### 🔑 关键超参数
- **rank (r)**：控制 LoRA 表达能力，推荐 8-64
- **alpha (α)**：缩放因子，通常设为 2×rank
- **学习率**：LoRA 微调推荐 1e-4 到 1e-3
- **训练轮数**：通常 3-5 轮即可收敛

### 📊 预期效果
- **参数效率**：相比全量微调减少 95%+ 可训练参数
- **训练速度**：显存占用降低，训练时间缩短
- **部署便利**：LoRA 文件仅数MB，支持多任务切换
- **性能保持**：在垃圾短信分类任务上可达 95%+ 准确率

## 🚀 进阶方向

### 🔬 深入研究
1. **QLoRA**：结合量化的更高效 LoRA 变体
2. **AdaLoRA**：自适应调整不同层的 rank 大小
3. **LoRA+**：改进的 LoRA 算法，更好的收敛性
4. **多任务 LoRA**：一个基座模型适配多个下游任务

### 🛠️ 实际应用
1. **领域适应**：将通用模型适配到特定领域（法律、医疗等）
2. **多语言支持**：为不同语言创建 LoRA 适配器
3. **个性化服务**：为不同用户群体训练专属 LoRA
4. **模型压缩**：结合剪枝、量化等技术进一步优化

### 📚 学习资源
1. **论文阅读**：LoRA原论文及相关工作
2. **开源项目**：PEFT、LLaMA-Factory、Alpaca-LoRA
3. **实践项目**：尝试在更大数据集和模型上应用
4. **社区交流**：参与开源社区，分享经验心得

---

## 💡 最后的建议

1. **循序渐进**：先在小模型小数据上验证，再扩展到大规模
2. **参数调优**：根据具体任务调整 rank 和 alpha
3. **评估全面**：不仅看准确率，还要关注泛化能力
4. **版本控制**：记录超参数和结果，方便对比优化

🎉 **恭喜你完成了 LoRA 微调的完整学习之旅！现在你已经具备了将理论应用到实际项目中的能力。**
